In [49]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
import json

In [50]:
g = Graph()
g.parse("food_nutrition.ttl", format="turtle")

<Graph identifier=Ned416a2456814b63b31625b5f3d80f3d (<class 'rdflib.graph.Graph'>)>

In [ ]:
queries = {
    "landing_page_groups.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT DISTINCT ?group
        WHERE {
          ?food a fn:Food ;
                fn:belongsToFoodWheelGroup ?group .
        }
        ORDER BY ?group
    """,

    "subgroups_overview.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT DISTINCT ?group ?subgroup
        WHERE {
          ?food a fn:Food ;
                fn:belongsToFoodWheelGroup ?group ;
                fn:belongsToFoodWheelSubGroup ?subgroup .
        }
        ORDER BY ?group ?subgroup
    """,

    "subpage_vegetables.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?diet ?benefit 
               ?protein ?proteinUnit ?carbs ?carbsUnit ?fat ?fatUnit 
               ?vitaminC ?vitaminCUnit ?iron ?ironUnit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:belongsToFoodWheelSubGroup fn:Vegetables .
          OPTIONAL { ?food fn:belongsToDiet ?diet }
          OPTIONAL { ?food fn:hasBenefit ?benefit }
          OPTIONAL { ?food fn:hasProtein ?protein }
          OPTIONAL { ?food fn:hasProteinUnit ?proteinUnit }
          OPTIONAL { ?food fn:hasCarbohydrates ?carbs }
          OPTIONAL { ?food fn:hasCarbohydratesUnit ?carbsUnit }
          OPTIONAL { ?food fn:hasFat ?fat }
          OPTIONAL { ?food fn:hasFatUnit ?fatUnit }
          OPTIONAL { ?food fn:hasVitaminC ?vitaminC }
          OPTIONAL { ?food fn:hasVitaminCUnit ?vitaminCUnit }
          OPTIONAL { ?food fn:hasIron ?iron }
          OPTIONAL { ?food fn:hasIronUnit ?ironUnit }
        }
        ORDER BY DESC(?vitaminC)
        LIMIT 30
    """,

    "top_vitaminC.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?vitaminC ?vitaminCUnit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:hasVitaminC ?vitaminC ;
                fn:hasVitaminCUnit ?vitaminCUnit .
          FILTER(?vitaminC > 0)
        }
        ORDER BY DESC(?vitaminC)
        LIMIT 10
    """,

    "diet_filters_vegetarian.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?group ?subgroup 
               ?protein ?proteinUnit ?fat ?fatUnit ?carbs ?carbsUnit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:belongsToDiet ?diet ;
                fn:belongsToFoodWheelGroup ?group ;
                fn:belongsToFoodWheelSubGroup ?subgroup .
          OPTIONAL { ?food fn:hasProtein ?protein }
          OPTIONAL { ?food fn:hasProteinUnit ?proteinUnit }
          OPTIONAL { ?food fn:hasFat ?fat }
          OPTIONAL { ?food fn:hasFatUnit ?fatUnit }
          OPTIONAL { ?food fn:hasCarbohydrates ?carbs }
          OPTIONAL { ?food fn:hasCarbohydratesUnit ?carbsUnit }
          FILTER(CONTAINS(STR(?diet), "Vegetarian"))
        }
        ORDER BY DESC(?protein)
        LIMIT 25
    """,

    "foods_potassium.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?benefit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:hasBenefit ?benefit .
          FILTER(CONTAINS(STR(?benefit), "Potassium"))
        }
        ORDER BY ?foodName
    """,

    "all_foods.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?group ?subgroup ?diet ?benefit 
               ?calories ?caloriesUnit 
               ?fat ?fatUnit 
               ?iron ?ironUnit 
               ?protein ?proteinUnit 
               ?vitaminB12 ?vitaminB12Unit 
               ?vitaminC ?vitaminCUnit 
               ?vitaminD ?vitaminDUnit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:belongsToFoodWheelGroup ?group ;
                fn:belongsToFoodWheelSubGroup ?subgroup .
          OPTIONAL { ?food fn:belongsToDiet ?diet }
          OPTIONAL { ?food fn:hasBenefit ?benefit }
          OPTIONAL { ?food fn:hasCalories ?calories }
          OPTIONAL { ?food fn:hasCaloriesUnit ?caloriesUnit }
          OPTIONAL { ?food fn:hasFat ?fat }
          OPTIONAL { ?food fn:hasFatUnit ?fatUnit }
          OPTIONAL { ?food fn:hasIron ?iron }
          OPTIONAL { ?food fn:hasIronUnit ?ironUnit }
          OPTIONAL { ?food fn:hasProtein ?protein }
          OPTIONAL { ?food fn:hasProteinUnit ?proteinUnit }
          OPTIONAL { ?food fn:hasVitaminB12 ?vitaminB12 }
          OPTIONAL { ?food fn:hasVitaminB12Unit ?vitaminB12Unit }
          OPTIONAL { ?food fn:hasVitaminC ?vitaminC }
          OPTIONAL { ?food fn:hasVitaminCUnit ?vitaminCUnit }
          OPTIONAL { ?food fn:hasVitaminD ?vitaminD }
          OPTIONAL { ?food fn:hasVitaminDUnit ?vitaminDUnit }
        }
        ORDER BY ?group ?subgroup ?foodName
    """
}

In [52]:
def row_to_dict(row):
    result = {}
    for k, v in row.asdict().items():
        if v is None:
            result[k] = None
        else:
            # Handle RDF URIRefs
            if isinstance(v, URIRef):
                # Remove namespace for readability
                result[k] = str(v).split("#")[-1]
            # Handle RDF Literals
            elif hasattr(v, "datatype") and v.datatype:
                # Keep value as string to preserve units (like "36 mg")
                result[k] = str(v)
            else:
                # fallback: just convert to string
                result[k] = str(v)
    return result

In [53]:
### Run queries and save results
for filename, query in queries.items():
    print(f"Running query -> {filename}")
    results = g.query(query)
    data = [row_to_dict(row) for row in results]

    print(f"Rows: {len(data)}")

    ### Print first 5 results (to avoid flooding the terminal)
    for d in data[:5]:
        print(json.dumps(d, indent=2, ensure_ascii=False))
    
    if len(data) > 5:
        print(f"... ({len(data)-5} more rows)")


with open("all_foods.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"Saved {len(data)} rows to {filename}")

Running query -> landing_page_groups.json
Rows: 5
{
  "group": "Bread%2C%20grain%2Fcereal%20products%20and%20potatoes"
}
{
  "group": "Dairy%2C%20nuts%2C%20fish%2C%20legumes%2C%20meat%20and%20eggs"
}
{
  "group": "Drinks"
}
{
  "group": "Spreading%20and%20cooking%20fats"
}
{
  "group": "Vegetables%20and%20fruit"
}
Running query -> subgroups_overview.json
Rows: 8
{
  "group": "Bread%2C%20grain%2Fcereal%20products%20and%20potatoes",
  "subgroup": "Bread%2C%20grain%2Fcereal%20products%20and%20potatoes"
}
{
  "group": "Dairy%2C%20nuts%2C%20fish%2C%20legumes%2C%20meat%20and%20eggs",
  "subgroup": "Dairy"
}
{
  "group": "Dairy%2C%20nuts%2C%20fish%2C%20legumes%2C%20meat%20and%20eggs",
  "subgroup": "Fish%2C%20legumes%2C%20meat%20and%20eggs"
}
{
  "group": "Dairy%2C%20nuts%2C%20fish%2C%20legumes%2C%20meat%20and%20eggs",
  "subgroup": "Nuts"
}
{
  "group": "Drinks",
  "subgroup": "Drinks"
}
... (3 more rows)
Running query -> subpage_vegetables.json
Rows: 30
{
  "foodName": "Brussel sprouts raw"